In [1]:
from openff.qcsubmit.common_structures import QCSpec, SCFProperties
from openff.qcsubmit.factories import BasicDatasetFactory
import numpy as np
from qcelemental.models.results import WavefunctionProtocolEnum
from qcportal.models.common_models import DriverEnum

In [2]:
factory = BasicDatasetFactory(driver=DriverEnum.gradient,
                              qc_specifications={'default': QCSpec(method='wb97m-d3bj', 
                                                                   basis='def2-tzvppd', 
                                                                   program='psi4', 
                                                                   spec_name='spice_default', 
                                                                   spec_description='SPICE quantum chemistry specification',
                                                                   store_wavefunction=WavefunctionProtocolEnum.orbitals_and_eigenvalues, 
                                                                   implicit_solvent=None, 
                                                                   maxiter=200, 
                                                                   scf_properties=[SCFProperties.Dipole, SCFProperties.Quadrupole, SCFProperties.WibergLowdinIndices, 
                                                                                   SCFProperties.MayerIndices, SCFProperties.MBISCharges],
                                                                   keywords=None)},
                             store_wavefunction=WavefunctionProtocolEnum.orbitals_and_eigenvalues)

In [3]:
dataset = factory.create_dataset(dataset_name="SPICE DES370K single points", molecules="des370K.hdf5", tagline="QM dataset for ML", description="Data source: https://github.com/openmm/qmdataset/tree/main/des370k")

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 1, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 0, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 2, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 6, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 13, aromatic: False, chiral: False
Atom atomic num: 6, name: , idx: 16, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 15, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 17, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 21, aromatic: False, chiral: False
bond ord

ValidationError: 1 validation error for Molecule
connectivity
  ensure this value has at least 1 items (type=value_error.list.min_items; limit_value=1)

In [ ]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

In [ ]:
dataset.visualize("des370k.pdf")

In [ ]:
dataset.molecules_to_file("des370k.smi", "smi")

In [ ]:
dataset.metadata.submitter = 'Josh Horton/Pavan Behara/David Dotson'

In [ ]:
dataset.metadata

In [ ]:
dataset.export_dataset("dataset.json.bz2")